In [ ]:
%cd /app

import os
import pandas as pd
import datetime
import numpy as np
import lightgbm as lgb

## Format CFTC Data

In [ ]:
def ltrim(n):
    if n[:2] == '00':
        n = n[2:]
    if n[:1] == '0':
        n = n[1:]
    
    return n


# WTI contract market code: 67651

In [ ]:
selected_columns = [
    "Report_Date_as_MM_DD_YYYY",
   # "CFTC_Contract_Market_Code",
    "M_Money_Positions_Long_ALL",
    "M_Money_Positions_Short_ALL",
    "M_Money_Positions_Spread_ALL"
]

dfs = []
for i in range(15, 25):
    print(i)
    i = str(i)
    df = pd.read_excel(f"data/cftc_data/f_{i}.xls", 
        sheet_name="XLS", 
        #usecols=selected_columns,
    )
    dfs.append(df)
dfs = pd.concat(dfs)
dfs = dfs.loc[dfs['CFTC_Contract_Market_Code'].str.contains("67651"), selected_columns]

dfs.columns = ['date', 'mm_long', 'mm_short', 'mm_spread']
dfs.set_index("date", inplace=True)
dfs.sort_index(inplace=True)
dfs['mm_net'] = dfs['mm_long'] - dfs['mm_short']
dfs['mm_long'].plot()
mm_df = dfs

In [ ]:
dfs.to_csv("data/wti_mm_positioning.csv")

## Prices and Spreads

In [ ]:
df_rbob = pd.read_excel("data/rbob_price.xls", sheet_name = "Data 1", index_col="date")
df_ho = pd.read_excel("data/heating_oil_price.xls", sheet_name="Data 1", index_col="date")
df_wti = pd.read_excel("data/wti_cushing_price.xls", sheet_name="Data 1", index_col="date")

In [ ]:
price_df = pd.concat([df_rbob, df_wti, df_ho], axis=1, join="inner")

In [ ]:
price_df['wti_cush_spot'].plot()

In [ ]:
price_df['321_spread'] = (2/3)*price_df['rbob_m1']*42 + (1/3)*price_df['ho_m1']*42 - price_df['wti_cush_spot']

In [ ]:
price_df.to_csv("data/prices.csv")

## Balance Data

In [ ]:
exports_df = pd.read_excel("data/us_crude_exports.xls", sheet_name="Data 1", index_col=0)
imports_df = pd.read_excel("data/us_crude_import.xls", sheet_name="Data 1", index_col=0)
production_df = pd.read_excel("data/us_crude_production.xls", sheet_name="Data 1", index_col=0)
cushing_df = pd.read_excel("data/weekly_cushing_stocks.xls", sheet_name="Data 1", index_col=0)
total_stocks_df = pd.read_excel("data/weekly_petroleum_stocks.xls", sheet_name="Data 1", index_col=0)

rename_stocks = {
    "WCRSTUS1":"total_us_inc_spr",
    "WCSSTUS1":"spr"
}

total_stocks_df.rename(mapper=rename_stocks, axis=1, inplace=True)

In [ ]:
balance_df = pd.concat([exports_df, imports_df, production_df, cushing_df, total_stocks_df], axis=1, join="inner")
balance_df['total_us_ex_spr'] = balance_df['total_us_inc_spr'] - balance_df['spr']

## Macro Data

In [ ]:
spx_df = pd.read_excel("data/s_and_p_500.xls", index_col=0)
two_yr_trea_df = pd.read_csv("data/two_year_treas_yield.csv", index_col=0, parse_dates=True)
macro_df = spx_df.join(two_yr_trea_df)


In [ ]:
# MM is Tuesday
# stocks is Fridaycombined_df['treas_2yr'] = combined_df['treas_2yr'].astype(float)
#mm_df
#balance_df

# align to Tuesday

In [ ]:
combined_df = pd.merge(
    mm_df, balance_df, left_index=True, right_index=True, how = "outer"
)
combined_df = combined_df.loc[combined_df.index >= mm_df.index.min()]
combined_df[['crude_exports', 'crude_imports', 'total_production', 'cushing_stocks', 'total_us_inc_spr', 'spr']] = combined_df[['crude_exports', 'crude_imports', 'total_production', 'cushing_stocks', 'total_us_inc_spr', 'spr']].interpolate(method="linear")
combined_df = combined_df.loc[combined_df.index.isin(mm_df.index), ['mm_long', 'mm_short', 'mm_net', 'crude_exports', 'crude_imports', 'total_production', 'cushing_stocks',  'total_us_inc_spr', 'spr']]
combined_df = combined_df.join(price_df).join(macro_df)
combined_df = combined_df.loc[(combined_df.index <= combined_df['wti_cush_spot'].last_valid_index()) & (combined_df.index >= combined_df['treas_2yr'].first_valid_index())]
combined_df['treas_2yr'] = combined_df['treas_2yr'].astype(float)

In [ ]:
levels = ['cushing_stocks', 'total_us_inc_spr', 'wti_cush_spot', '321_spread']
levels_df = combined_df[levels].copy()

In [ ]:
final_df = combined_df.diff().join(levels_df, rsuffix="_level")
final_df['time_index'] = np.arange(0, len(final_df))

In [ ]:
final_df.iloc[1:].to_csv("data/final.csv")